In [1]:
###### Understanding intents and entities

In [4]:
bot_template = "BOT : {0}"
user_template = "USER : {0}"

# Define a function that responds to a user's message: respond
def respond(message):
    # Concatenate the user's message to the end of a standard bot respone
    bot_message = "I can hear you! You said: " + message
    # Return the result
    return bot_message

In [5]:
# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))

# Send a message to the bot
send_message("hello")

USER : hello
BOT : I can hear you! You said: hello


In [12]:
import re
keywords = {
            'greet': ['hello', 'hi', 'hey'],
            'thankyou': ['thank', 'thx'],
            'goodbye': ['bye', 'farewell']
           }
# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent, value in keywords.items():
    # Create regular expressions and compile them into pattern objects
    patterns[intent] = re.compile('|'.join(value))
    
# Print the patterns
print(patterns)

{'greet': re.compile('hello|hi|hey'), 'thankyou': re.compile('thank|thx'), 'goodbye': re.compile('bye|farewell')}


In [10]:
responses = {'greet': 'Hello you! :)', 
             'thankyou': 'you are very welcome', 
             'default': 'default message', 
             'goodbye': 'goodbye for now'
            }

# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            matched_intent = intent
    return matched_intent

# Define a respond function
def respond(message):
    # Call the match_intent function
    intent = match_intent(message)
    # Fall back to the default response
    key = "default"
    if intent in responses:
        key = intent
    return responses[key]

# Send messages
send_message("hello!")
send_message("bye byeee")
send_message("thanks very much!")

USER : hello!
BOT : Hello you! :)
USER : bye byeee
BOT : goodbye for now
USER : thanks very much!
BOT : you are very welcome


In [8]:
# Define find_name()
def find_name(message):
    name = None
    # Create a pattern for checking if the keywords occur
    name_keyword = re.compile(r"(name|call)")
    # Create a pattern for finding capitalized words
    name_pattern = re.compile('[A-Z]{1}[a-z]*')
    if name_keyword.search(message):
        # Get the matching words in the string
        name_words = name_pattern.findall(message)
        if len(name_words) > 0:
            # Return the name if the keywords are present
            name = ' '.join(name_words)
    return name

# Define respond()
def respond(message):
    # Find the name
    name = find_name(message)
    if name is None:
        return "Hi there!"
    else:
        return "Hello, {0}!".format(name)

# Send messages
send_message("my name is David Copperfield")
send_message("call me Ishmael")
send_message("people call me Cassandra")

USER : my name is David Copperfield
BOT : Hello, David Copperfield!
USER : call me Ishmael
BOT : Hello, Ishmael!
USER : people call me Cassandra
BOT : Hello, Cassandra!


In [6]:
###### Word vectors

In [2]:
import spacy
import numpy as np
sentences = [' i want to fly from boston at 838 am and arrive in denver at 1110 in the morning', 
 ' what flights are available from pittsburgh to baltimore on thursday morning', 
 ' what is the arrival time in san francisco for the 755 am flight leaving washington', 
 ' cheapest airfare from tacoma to orlando', 
 ' round trip fares from pittsburgh to philadelphia under 1000 dollars', 
 ' i need a flight tomorrow from columbus to minneapolis', 
 ' what kind of aircraft is used on a flight from cleveland to dallas', 
 ' show me the flights from pittsburgh to los angeles on thursday', 
 ' all flights from boston to washington', 
 ' what kind of ground transportation is available in denver', 
 ' show me the flights from dallas to san francisco', 
 ' show me the flights from san diego to newark by way of houston', 
 ' what is the cheapest flight from boston to bwi', 
 ' all flights to baltimore after 6 pm', 
 ' show me the first class fares from boston to denver', 
 ' show me the ground transportation in denver', 
 ' all flights from denver to pittsburgh leaving after 6 pm and before 7 pm', 
 ' i need information on flights for tuesday leaving baltimore for dallas dallas to boston and boston to baltimore', 
 ' please give me the flights from boston to pittsburgh on thursday of next week', 
 ' i would like to fly from denver to pittsburgh on united airlines', 
 ' show me the flights from san diego to newark', 
 ' please list all first class flights on united from denver to baltimore', 
 ' what kinds of planes are used by american airlines', 
 " i'd like to have some information on a ticket from denver to pittsburgh and atlanta", 
 " i'd like to book a flight from atlanta to denver", 
 ' which airline serves denver pittsburgh and atlanta', 
 " show me all flights from boston to pittsburgh on wednesday of next week which leave boston after 2 o'clock pm", 
 ' atlanta ground transportation', ' i also need service from dallas to boston arriving by noon', 
 ' show me the cheapest round trip fare from baltimore to dallas']

In [3]:
# Load the spacy model: nlp, en_core_web_md
nlp = spacy.load('en_core_web_md')

# Calculate the length of sentences
n_sentences = len(sentences)

# Calculate the dimensionality of nlp
embedding_dim = nlp.vocab.vectors_length

# Initialize the array with zeros: X
X = np.zeros((n_sentences, embedding_dim))

# Iterate over the sentences
for idx, sentence in enumerate(sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X[idx, :] = doc.vector

C:\Users\10231\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_md' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [18]:
#### Intent classification with sklearn

In [19]:
import pandas as pd
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')['label']
y_test = pd.read_csv('y_test.csv')['label']

In [20]:
# Import SVC
from sklearn.svm import SVC

# Create a support vector classifier
clf = SVC()

# Fit the classifier using the training data
clf.fit(X_train,y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Count the number of correct predictions
n_correct = 0
for i in range(len(y_test)):
    if y_pred[i] == y_test[i]:
        n_correct += 1
        
print("Predicted {0} correctly out of {1} test examples.".format(n_correct, len(y_test)))

C:\Users\10231\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Predicted 162 correctly out of 201 test examples.


In [21]:
#### Entity extraction

In [23]:
## Using spaCy's entity recogniser

# Define included entities
include_entities = ['DATE', 'ORG', 'PERSON']

# Define extract_entities()
def extract_entities(message):
    # Create a dict to hold the entities
    ents = dict.fromkeys(include_entities)
    # Create a spacy document
    doc = nlp(message)
    for ent in doc.ents:
        if ent.label_ in include_entities:
            # Save interesting entities
            ents[ent.label_] = ent.text
    return ents

print(extract_entities('friends called Mary who have worked at Google since 2010'))
print(extract_entities('people who graduated from MIT in 1999'))

{'DATE': '2010', 'ORG': 'Google', 'PERSON': 'Mary'}
{'DATE': '1999', 'ORG': 'MIT', 'PERSON': None}


In [ ]:
## Assigning roles using spaCy's parser
def entity_type(word):
    _type = None
    if word.text in colors:
        _type = "color"
    elif word.text in items:
        _type = "item"
    return _type

colors = ['black', 'red', 'blue']
items = ['shoes', 'handback', 'jacket', 'jeans']

In [ ]:
## Assigning roles using spaCy's parser

# Create the document
doc = ____

# Iterate over parents in parse tree until an item entity is found
def find_parent_item(word):
    # Iterate over the word's ancestors
    for parent in ____:
        # Check for an "item" entity
        if entity_type(____) == "____":
            return parent.text
    return None

# For all color entities, find their parent item
def assign_colors(doc):
    # Iterate over the document
    for word in ____:
        # Check for "color" entities
        if entity_type(word) == "____":
            # Find the parent
            item =  ____
            print("item: {0} has color : {1}".format(item, word))

# Assign the colors
____ 

In [ ]:
#### Robust NLU with Rasa

In [ ]:
## Rasa NLU

In [24]:
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config


# Create a trainer
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('demo-rasa.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

# Try it out
print(interpreter.parse("I'm looking for a Mexican restaurant in the North of town"))

C:\Users\10231\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_md' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
C:\Users\10231\AppData\Roaming\Python\Python37\site-packages\rasa_nlu\training_data\training_data.py:176: UserWarning: Intent 'None' has only 1 training examples! Minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_INTENT))
C:\Users\10231\AppData\Roaming\Python\Python37\site-packages\rasa_nlu\training_data\training_data.py:184: UserWarning: Entity 'area' has only 1 training examples! minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_ENTITY))


Fitting 2 folds for each of 6 candidates, totalling 12 fits
{'intent': {'name': 'restaurant_search', 'confidence': 0.6613460144663318}, 'entities': [{'start': 18, 'end': 25, 'value': 'mexican', 'entity': 'cuisine', 'confidence': 0.652738841752258, 'extractor': 'CRFEntityExtractor'}, {'start': 44, 'end': 49, 'value': 'north', 'entity': 'location', 'confidence': 0.840130232993451, 'extractor': 'CRFEntityExtractor'}], 'intent_ranking': [{'name': 'restaurant_search', 'confidence': 0.6613460144663318}, {'name': 'goodbye', 'confidence': 0.11494089149590193}, {'name': 'location', 'confidence': 0.06706916189156273}, {'name': 'hotel_search', 'confidence': 0.060250658807240266}, {'name': 'affirm', 'confidence': 0.05194592653717817}, {'name': 'greet', 'confidence': 0.04444734680178499}], 'text': "I'm looking for a Mexican restaurant in the North of town"}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [ ]:
## Data-efficient entity recognition

In [25]:
print(interpreter.parse("show me Chinese food in the centre of town"))
print(interpreter.parse("I want an Indian restaurant in the west"))
print(interpreter.parse("are there any good pizza places in the center?"))

{'intent': {'name': 'restaurant_search', 'confidence': 0.6275842256508134}, 'entities': [{'start': 8, 'end': 15, 'value': 'chinese', 'entity': 'cuisine', 'confidence': 0.6057065130162513, 'extractor': 'CRFEntityExtractor', 'processors': ['EntitySynonymMapper']}, {'start': 28, 'end': 34, 'value': 'centre', 'entity': 'location', 'confidence': 0.9343784028432665, 'extractor': 'CRFEntityExtractor'}], 'intent_ranking': [{'name': 'restaurant_search', 'confidence': 0.6275842256508134}, {'name': 'goodbye', 'confidence': 0.11703208416583816}, {'name': 'location', 'confidence': 0.07739500832647572}, {'name': 'affirm', 'confidence': 0.06591564824023495}, {'name': 'greet', 'confidence': 0.056273525245450276}, {'name': 'hotel_search', 'confidence': 0.055799508371187384}], 'text': 'show me Chinese food in the centre of town'}
{'intent': {'name': 'restaurant_search', 'confidence': 0.7014494974447648}, 'entities': [{'start': 10, 'end': 16, 'value': 'indian', 'entity': 'cuisine', 'confidence': 0.801008

In [ ]:
## Assigning roles using spaCy's parser

# Create the document
doc = nlp("Let us see that jacket in red and some blue jeans!")

# Iterate over parents in parse tree until an item entity is found
def find_parent_item(word):
    # Iterate over the word's ancestors
    for parent in list(word.ancestors):
        # Check for an "item" entity
        if entity_type(parent) == "item":
            return parent.text
    return None

# For all color entities, find their parent item
def assign_colors(doc):
    # Iterate over the document
    for word in doc:
        # Check for "color" entities
        if entity_type(word) == "color":
            # Find the parent
            item =  find_parent_item(word)
            print("item: {0} has color : {1}".format(item, word))

# Assign the colors
assign_colors(doc) 

In [ ]:
## Assigning roles using spaCy's parser

# Create the document
doc = nlp("I like to wear a blue jacket and a red jean.")
# Iterate over parents in parse tree until an item entity is found
def find_parent_item(word):
    # Iterate over the word's ancestors
    for parent in word.ancestors:
        # Check for an "item" entity
        if entity_type(parent) == "item":
            return parent.text
    return None

# For all color entities, find their parent item
def assign_colors(doc):
    # Iterate over the document
    for word in doc:
        # Check for "color" entities
        if entity_type(word) == "color":
            # Find the parent
            item =  find_parent_item(word)
            print("item: {0} has color : {1}".format(item, word))

# Assign the colors
assign_colors(doc)